In [1]:
!pip install --upgrade pip --user
!pip install tfx==1.2.0 alibi==0.6.0 boto3==1.20.5 keras-tuner pandas kfp==1.7.2 --user --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 108.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.1
    Uninstalling pip-22.1:
      Successfully uninstalled pip-22.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Using cached keras_tuner-1.1.2-py3-none-any.whl (133 kB)


In [2]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma

from tfx import v1 as tfx
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.components.base import executor_spec
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# TensorFlowの警告出力を抑止する
import logging
import warnings
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

In [3]:
os.environ['UNSUPPORTED_DO_NOT_PACKAGE_USER_MODULES'] = 'True'

In [4]:
context = InteractiveContext()

In [5]:
header = [
    'age','workclass','id','education','education-num','marital-status',
    'occupation','relationship','race','gender','capital-gain','capital-loss',
    'hours-per-week','native-country','income'
]

df = pd.read_csv('adult.data', names=header, delimiter=', ', usecols=lambda x: x not in ['id', 'race', 'native-country'])

In [6]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def create_tf_example(features, label):
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'age': _int64_feature(features[0]),
        'workclass': _bytes_feature(features[1].encode('utf-8')),
        'education': _bytes_feature(features[2].encode('utf-8')),
        'education-num': _int64_feature(features[3]),
        'marital-status': _bytes_feature(features[4].encode('utf-8')),
        'occupation': _bytes_feature(features[5].encode('utf-8')),
        'relationship': _bytes_feature(features[6].encode('utf-8')),
        'gender': _bytes_feature(features[7].encode('utf-8')),
        'capital-gain': _int64_feature(features[8]),
        'capital-loss': _int64_feature(features[9]),
        'hours-per-week': _int64_feature(features[10]),
        'income': _bytes_feature(label.encode('utf-8')),
    }))
    return tf_example

In [7]:
tfrecord_data_root = os.path.join(os.getcwd(), 'tfrecord_data', 'span-1', 'v1')
tfrecord_data_path = os.path.join(tfrecord_data_root, 'adult.span-1.tfrecords')
os.makedirs(tfrecord_data_root, exist_ok=True)

with tf.io.TFRecordWriter(tfrecord_data_path) as writer:
    for row in df.values:
        features, label = row[:-1], row[-1]
        example = create_tf_example(features, label)
        writer.write(example.SerializeToString())

In [8]:
import boto3
from botocore.client import Config

os.environ['AWS_ACCESS_KEY_ID'] = 'XJGhECgFUmY9vtcK'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'Uyz8YLhpy3PAbDCSXIRFnTM1HPQ7K4Ib'
os.environ["AWS_REGION"] = 'ap-northeast-1'
os.environ["S3_ENDPOINT"] = "http://minio.default.svc.cluster.local"

s3_resource = boto3.resource('s3',
                endpoint_url = os.environ["S3_ENDPOINT"],
                aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID'],
                aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY'],
                region_name = os.environ["AWS_REGION"],
                config = Config(signature_version='s3v4'))

In [9]:
bucket_name = 'census-income'
bucket_name_s3_prefix = 's3://%s' % bucket_name

try:
    bucket = s3_resource.Bucket(bucket_name)
    bucket.create()
    print("Create S3 bucket:", bucket_name)
except:
    print("Exsiting S3 bucket:", bucket_name)

Exsiting S3 bucket: census-income


In [10]:
bucket.upload_file(tfrecord_data_path, 'data/span-1/v1/adult.span-1.tfrecords')

In [11]:
beam_pipeline_args = ['--s3_access_key_id=%s' % os.environ['AWS_ACCESS_KEY_ID'],
                '--s3_secret_access_key=%s' % os.environ['AWS_SECRET_ACCESS_KEY'],
                '--s3_endpoint_url=%s' % os.environ["S3_ENDPOINT"],
                '--s3_region_name=%s' % os.environ["AWS_REGION"],
                '--s3_verify=0',
                '--s3_disable_ssl',
                '--direct_num_workers=1']

context.beam_pipeline_args=beam_pipeline_args

In [12]:
example_gen_input = {"splits": [{"name": "single_split",
                "pattern": "span-{SPAN}/v{VERSION}/*"}]
                    }
example_gen_output = {"split_config": {"splits": [{"name": "train", "hash_buckets": 2},
                {"name": "eval", "hash_buckets": 1}]
                                      }}
example_gen = tfx.components.ImportExampleGen(input_base = '%s/data/' % bucket_name_s3_prefix,
                input_config = example_gen_input,
                output_config = example_gen_output)

context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/ImportExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:10601607,xor_checksum:1653282886,sum_checksum:1653282886"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        custom_properties {
          key: "version"
          value {
            int_value: 1
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
example_gen_input = {"splits": [{"name": "single_split",
                                 "pattern": "span-{SPAN}/v{VERSION}/*"}]
                    }

In [14]:
example_gen_output_three_division = {"split_config": {"splits": [{"name": "train", "hash_buckets": 6},
                                {"name": "eval", "hash_buckets": 2},
                                {"name": "test", "hash_buckets": 2}]}}
example_gen_three_division = tfx.components.ImportExampleGen(input_base = '%s/data/' % bucket_name_s3_prefix,
                                input_config = example_gen_input,
                                output_config = example_gen_output_three_division)
context.run(example_gen_three_division)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 2
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 2
        type_id: 14
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/ImportExampleGen/examples/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:10601607,xor_checksum:1653282886,sum_checksum:1653282886"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        custom_properties {
          key: "version"
          value {
            int_value: 1
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [15]:
statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 3
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 3
        type_id: 16
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/StatisticsGen/statistics/3"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [16]:
context.show(statistics_gen.outputs['statistics'])

In [17]:
schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 4
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 18
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/SchemaGen/schema/4"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [18]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'capital-gain',INT,required,,-
'capital-loss',INT,required,,-
'education',STRING,required,,'education'
'education-num',INT,required,,-
'gender',STRING,required,,'gender'
'hours-per-week',INT,required,,-
'income',STRING,required,,'income'
'marital-status',STRING,required,,'marital-status'


,Values
Domain,
'education',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'gender',"'Female', 'Male'"
'income',"'<=50K', '>50K'"
'marital-status',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'occupation',"'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'relationship',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'workclass',"'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"


In [19]:
bucket.upload_file(os.path.join(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt'), 'schema/schema.pbtxt')

In [20]:
import_schema = tfx.dsl.Importer(
    source_uri = '%s/schema/' % bucket_name_s3_prefix,
    artifact_type = tfx.types.standard_artifacts.Schema).with_id('schema_importer')
context.run(import_schema)

ExecutionResult(
    component_id: schema_importer
    execution_id: 5
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 5
        type_id: 18
        uri: "s3://census-income/schema/"
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [21]:
import_schema = tfx.dsl.Importer(
    source_uri = '%s/schema/' % bucket_name_s3_prefix,
    artifact_type = tfx.types.standard_artifacts.Schema).with_id('schema_importer')
context.run(import_schema)

ExecutionResult(
    component_id: schema_importer
    execution_id: 6
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 5
        type_id: 18
        uri: "s3://census-income/schema/"
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        create_time_since_epoch: 1653282915309
        last_update_time_since_epoch: 1653282915309
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [22]:
# スキーマファイルの読み込み
from tensorflow_metadata.proto.v0 import schema_pb2

# スキーマファイルの読み込み
schema_output_file = os.path.join(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_output_file)

# スキーマの更新（有効範囲の設定）
tfdv.set_domain(schema, 'age', schema_pb2.IntDomain(name='age', min=30, max=60))
# スキーマの更新（ドメインの削除・追加）
workclass_domain = tfdv.get_domain(schema, 'workclass')
workclass_domain.value.remove('?')
workclass_domain.value.append('unknown_workclass')

# 更新したスキーマの確認
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,min: 30; max: 60
'capital-gain',INT,required,,-
'capital-loss',INT,required,,-
'education',STRING,required,,'education'
'education-num',INT,required,,-
'gender',STRING,required,,'gender'
'hours-per-week',INT,required,,-
'income',STRING,required,,'income'
'marital-status',STRING,required,,'marital-status'


,Values
Domain,
'education',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'gender',"'Female', 'Male'"
'income',"'<=50K', '>50K'"
'marital-status',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'occupation',"'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'relationship',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'workclass',"'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay', 'unknown_workclass'"


In [23]:
update_schema_output_dir = os.path.join(os.getcwd(), 'update_schema')
os.makedirs(update_schema_output_dir, exist_ok=True)
tfdv.write_schema_text(schema, os.path.join(update_schema_output_dir, 'schema.pbtxt'))

In [24]:
transform_module_file_name = 'adult_income_transorm_module.py'
transform_module_file_path = os.path.join(os.getcwd(), transform_module_file_name)

In [25]:
%%writefile {transform_module_file_path}
import os
import sys

import tensorflow as tf
import tensorflow_transform as tft

# 数値変数の項目名リスト
NUMERIC_FEATURE_KEYS = [
    'age',
    'education-num',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
]
# カテゴリー変数の項目名とその次元数の辞書
ONE_HOT_FEATURES = {'workclass': 8,
                    'education': 16,
                    'marital-status': 7,
                    'occupation': 14,
                    'relationship': 6,
                    'gender': 2
                   }
# 予測対象の項目名
LABEL_KEY = 'income'

# 変換後の項目名の変換用関数
def transformed_name(key):
    return key + '_xf'

Overwriting /home/jovyan/resource/adult_income_transorm_module.py


In [26]:
%%writefile -a {transform_module_file_path}

def preprocessing_fn(inputs):
    #文字列型の値を数値型に変換する関数
    def __convert_string2int_value(key, original_values, converted_values):
        initializer = tf.lookup.KeyValueTensorInitializer(
            keys=original_values,
            values=tf.cast(converted_values, tf.int64),
            key_dtype=tf.string,
            value_dtype=tf.int64)
        table = tf.lookup.StaticHashTable(initializer, default_value=-1)
        return table.lookup(inputs[key])

    outputs = {}
    #欠損値を最頻値に変換
    inputs['workclass'] = tf.where(tf.equal(inputs['workclass'], '?'),
                                            'Private', inputs['workclass'])
    inputs['occupation'] = tf.where(tf.equal(inputs['occupation'], '?'),
                                             'Prof-specialty', inputs['occupation'])

    # TFTの関数を使って数値型の項目を正規化 (平均値0標準偏差1に変換)
    for key in NUMERIC_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.scale_to_z_score(inputs[key])

    # TFT の関数を使ってダミー変数化(One-Hot エンコーディング)
    for key in ONE_HOT_FEATURES.keys():
        dim = ONE_HOT_FEATURES[key]
        index = tft.compute_and_apply_vocabulary(
            tf.strings.strip(inputs[key]),
            num_oov_buckets=0,
            vocab_filename=key)
        one_hot_tensor = tf.one_hot(index, dim)
        one_hot_tensor = tf.reshape(one_hot_tensor, [-1, dim])
        for i in range(0, dim):
            one_hot_name = transformed_name(key + '_' + str(i))
            outputs[one_hot_name] = one_hot_tensor[:,i]
    # Labelを0と1に変換
    table_keys = ['<=50K', '>50K']
    outputs[transformed_name(LABEL_KEY)] = __convert_string2int_value(
             LABEL_KEY,  table_keys, [0, 1])

    return outputs

Appending to /home/jovyan/resource/adult_income_transorm_module.py


In [27]:
bucket.upload_file(transform_module_file_path, 'module/%s' % transform_module_file_name)

In [28]:
param_transform_module_file = '%s/module/%s' % (bucket_name_s3_prefix, transform_module_file_name)

transform = tfx.components.Transform(
    examples = example_gen.outputs['examples'],
    schema = import_schema.outputs['result'],
    module_file = param_transform_module_file)
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 6
        type_id: 21
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Transform/transform_graph/7"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Transform/transformed_examples/7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 8
        type_id: 22
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Transform/updated_analyzer_cache/7"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 9
        type_id: 18
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Transform/pre_transform_schema/7"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"


In [29]:
import_anomalies_schema = tfx.dsl.Importer(
    source_uri = update_schema_output_dir,
    artifact_type = tfx.types.standard_artifacts.Schema).with_id('schema_importer')
context.run(import_anomalies_schema)

ExecutionResult(
    component_id: schema_importer
    execution_id: 8
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 14
        type_id: 18
        uri: "/home/jovyan/resource/update_schema"
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [30]:
anomalies_example_validator = tfx.components.ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema = import_anomalies_schema.outputs['result'])
context.run(anomalies_example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 9
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 15
        type_id: 23
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/ExampleValidator/anomalies/9"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [31]:
context.show(anomalies_example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'age',Multiple errors,Unexpectedly small value: 17. Unexpectedly large value: 90.
'workclass',Unexpected string values,Examples contain values missing from the schema: ? (~5%).


,Anomaly short description,Anomaly long description
Feature name,,
'workclass',Unexpected string values,Examples contain values missing from the schema: ? (~5%).
'age',Multiple errors,Unexpectedly small value: 17. Unexpectedly large value: 90.


In [32]:
example_validator = tfx.components.ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema = import_schema.outputs['result'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 10
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 16
        type_id: 23
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/ExampleValidator/anomalies/10"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [33]:
context.show(example_validator.outputs['anomalies'])

In [34]:
context.show(transform.outputs['post_transform_schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age_xf',FLOAT,required,,-
'capital-gain_xf',FLOAT,required,,-
'capital-loss_xf',FLOAT,required,,-
'education-num_xf',FLOAT,required,,-
'education_0_xf',FLOAT,required,,-
'education_10_xf',FLOAT,required,,-
'education_11_xf',FLOAT,required,,-
'education_12_xf',FLOAT,required,,-
'education_13_xf',FLOAT,required,,-


In [35]:
bucket.upload_file(os.path.join(transform.outputs['post_transform_schema'].get()[0].uri, 'schema.pbtxt'), 'post_schema/schema.pbtxt')

In [36]:
%%writefile -a {transform_module_file_path}

def stats_options_updater_fn(stats_type, stats_options):
    import tensorflow_data_validation as tfdv
    from tfx.components.transform import stats_options_util

    if stats_type == stats_options_util.StatsType.POST_TRANSFORM:
        load_post_schema = tfdv.load_schema_text('s3://census-income/post_schema/schema.pbtxt')
        stats_options.schema = load_post_schema
    return stats_options

Appending to /home/jovyan/resource/adult_income_transorm_module.py


In [37]:
bucket.upload_file(transform_module_file_path, 'module/%s' % transform_module_file_name)

In [38]:
transformed_train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')
tfrecord_filenames = [os.path.join(transformed_train_uri, name)
                      for name in os.listdir(transformed_train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
#上から3行を表示
for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    print(example)

features {
  feature {
    key: "age_xf"
    value {
      float_list {
        value: 0.035811714828014374
      }
    }
  }
  feature {
    key: "capital-gain_xf"
    value {
      float_list {
        value: 0.1401928961277008
      }
    }
  }
  feature {
    key: "capital-loss_xf"
    value {
      float_list {
        value: -0.21797475218772888
      }
    }
  }
  feature {
    key: "education-num_xf"
    value {
      float_list {
        value: 1.1318093538284302
      }
    }
  }
  feature {
    key: "education_0_xf"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "education_10_xf"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "education_11_xf"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "education_12_xf"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "education_13_xf"
    value {
      float_list {
        

In [39]:
post_transform_path = os.path.join(transform.outputs['post_transform_stats'].get()[0].uri, 'FeatureStats.pb')
post_stats = tfdv.load_stats_binary(post_transform_path)
tfdv.visualize_statistics(post_stats)

In [40]:
trainer_module_without_tuning_file_name = 'adult_income_trainer_module_without_tuning.py'
trainer_module_without_tuning_file_path = os.path.join(os.getcwd(), trainer_module_without_tuning_file_name)

In [41]:
%%writefile {trainer_module_without_tuning_file_path}

import absl
import os
import sys

from typing import List, Text

import kerastuner
import tensorflow as tf
import tensorflow_transform as tft

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, concatenate

from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx.components.trainer.executor import TrainerFnArgs
from tfx_bsl.tfxio import dataset_options

NUMERIC_FEATURE_KEYS = [
    'age',
    'education-num',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
]
ONE_HOT_FEATURES = {'workclass': 8,
                    'education': 16,
                    'marital-status': 7,
                    'occupation': 14,
                    'relationship': 6,
                    'gender': 2
                   }
LABEL_KEY = 'income'

def transformed_name(key):
    return key + '_xf'

def create_model():
    model = Sequential()

    inputs = []
    for key in NUMERIC_FEATURE_KEYS:
        inputs.append(Input(shape=(1), name=transformed_name(key)))
    for key, dim in ONE_HOT_FEATURES.items():
        for i in range(0, dim):
            inputs.append(Input(shape=(1), name=transformed_name(key + '_' + str(i))))

    outputs = concatenate(inputs)
    units = 100
    outputs = Dense(units, activation='relu')(outputs)
    outputs = Dense(units, activation='relu')(outputs)
    outputs = Dense(1, activation='sigmoid')(outputs)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

Overwriting /home/jovyan/resource/adult_income_trainer_module_without_tuning.py


In [42]:
%%writefile -a {trainer_module_without_tuning_file_path}

def run_fn(fn_args: TrainerFnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    train_dataset = input_fn(fn_args.train_files, fn_args.data_accessor,
                                tf_transform_output, 40)
    eval_dataset = input_fn(fn_args.eval_files, fn_args.data_accessor,
                               tf_transform_output, 40)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = fn_args.model_run_dir, update_freq='batch'
    )

    model = create_model()
    model.fit(
        train_dataset,
        epochs = fn_args.custom_config["epoch"],
        steps_per_epoch = fn_args.train_steps,
        validation_data = eval_dataset,
        validation_steps = fn_args.eval_steps,
        callbacks = [tensorboard_callback]
    )

    signatures = {
        'serving_default':
            get_tf_examples_serving_signature(model, tf_transform_output),
        'transform_features':
            get_transform_features_signature(model, tf_transform_output),
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Appending to /home/jovyan/resource/adult_income_trainer_module_without_tuning.py


In [43]:
%%writefile -a {trainer_module_without_tuning_file_path}

def input_fn(file_pattern: List[Text],
             data_accessor: DataAccessor,
             tf_transform_output: tft.TFTransformOutput,
             batch_size: int = 200) -> tf.data.Dataset:
    return data_accessor.tf_dataset_factory(
        file_pattern,
        dataset_options.TensorFlowDatasetOptions(
            batch_size=batch_size, label_key=transformed_name(LABEL_KEY)),
        tf_transform_output.transformed_metadata.schema)

Appending to /home/jovyan/resource/adult_income_trainer_module_without_tuning.py


In [44]:
%%writefile -a {trainer_module_without_tuning_file_path}

def get_tf_examples_serving_signature(model, tf_transform_output):
    model.tft_layer_inference = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
    ])
    def serve_tf_examples_fn(serialized_tf_example):
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_feature_spec.pop(LABEL_KEY)
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
        transformed_features = model.tft_layer_inference(raw_features)

        outputs = model(transformed_features)
        return {'outputs': outputs}

    return serve_tf_examples_fn

Appending to /home/jovyan/resource/adult_income_trainer_module_without_tuning.py


In [45]:
%%writefile -a {trainer_module_without_tuning_file_path}

def get_transform_features_signature(model, tf_transform_output):
    model.tft_layer_eval = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
    ])
    def transform_features_fn(serialized_tf_example):
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
        transformed_features = model.tft_layer_eval(raw_features)
        return transformed_features

    return transform_features_fn

Appending to /home/jovyan/resource/adult_income_trainer_module_without_tuning.py


In [46]:
bucket.upload_file(trainer_module_without_tuning_file_path, 'module/%s' % trainer_module_without_tuning_file_name)

In [47]:
param_trainer_module_file = '%s/module/%s' % (bucket_name_s3_prefix, trainer_module_without_tuning_file_name)
param_trainer_train_args = {"num_steps": 100}
param_trainer_eval_args = {"num_steps": 50}
param_trainer_custom_config_args = {"epoch": 25}

trainer_without_tuner = tfx.components.Trainer(
    module_file = param_trainer_module_file,
    custom_executor_spec = executor_spec.ExecutorClassSpec(GenericExecutor),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = import_schema.outputs['result'],
    train_args = param_trainer_train_args,
    eval_args = param_trainer_eval_args,
    custom_config = param_trainer_custom_config_args)
context.run(trainer_without_tuner)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age_xf (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
education-num_xf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-gain_xf (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-loss_xf (InputLayer)    [(None, 1)]          0                                            
______________________________________________________________________________________________

ExecutionResult(
    component_id: Trainer
    execution_id: 11
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 17
        type_id: 26
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Trainer/model/11"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 18
        type_id: 27
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Trainer/model_run/11"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 27
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [48]:
tuner_module_file_name = 'adult_income_tuner_module.py'
tuner_module_file_path = os.path.join(os.getcwd(), tuner_module_file_name)

In [49]:
%%writefile {tuner_module_file_path}

import kerastuner
import tensorflow as tf
import tensorflow_transform as tft

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, concatenate

from tfx.components.trainer.fn_args_utils import FnArgs
from tfx.components.tuner.component import TunerFnResult

import adult_income_trainer_module_without_tuning as tm

def create_model(hparams: kerastuner.HyperParameters):
    model = Sequential()

    inputs = []
    for key in tm.NUMERIC_FEATURE_KEYS:
        inputs.append(Input(shape=(1), name=tm.transformed_name(key)))
    for key, dim in tm.ONE_HOT_FEATURES.items():
        for i in range(0, dim):
            inputs.append(Input(shape=(1), name=tm.transformed_name(key + '_' + str(i))))

    outputs = concatenate(inputs)
    units = hparams.get('units')
    outputs = Dense(units, activation='relu')(outputs)
    outputs = Dense(units, activation='relu')(outputs)
    outputs = Dense(1, activation='sigmoid')(outputs)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=hparams.get('optimizer'), loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    tuner = kerastuner.RandomSearch(
        create_model,
        max_trials = 10,
        hyperparameters = get_hyperparameters(),
        allow_new_entries = False,
        objective = kerastuner.Objective('val_accuracy', 'max'),
        directory = fn_args.working_dir,
        project_name = 'census-income-tuner'
    )

    tf_transform_graph = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_dataset = tm.input_fn(fn_args.train_files, fn_args.data_accessor,
                                tf_transform_graph, 40)
    eval_dataset = tm.input_fn(fn_args.eval_files, fn_args.data_accessor,
                               tf_transform_graph, 40)

    tb_callback = tf.keras.callbacks.TensorBoard(
        log_dir=fn_args.custom_config["tuner_log_dir"],
        update_freq='batch')

    return TunerFnResult(
        tuner = tuner,
        fit_kwargs = {
            'x': train_dataset,
            'validation_data': eval_dataset,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps,
            'callbacks': [tb_callback]
      })

Overwriting /home/jovyan/resource/adult_income_tuner_module.py


In [50]:
%%writefile -a {tuner_module_file_path}

def get_hyperparameters() -> kerastuner.HyperParameters:
    hp = kerastuner.HyperParameters()
    hp.Choice('optimizer', ['rmsprop', 'adam'], default='adam')
    hp.Int('units', min_value=100, max_value=250, step=50)
    return hp

Appending to /home/jovyan/resource/adult_income_tuner_module.py


In [51]:
import tempfile

tuner_tensorboard_log_dir = tempfile.mkdtemp()
param_tuner_custom_config_args = {"tuner_log_dir": tuner_tensorboard_log_dir}

param_tuner_train_args = {"num_steps": 100}
param_tuner_eval_args = {"num_steps": 50}

tuner = tfx.components.Tuner(
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = import_schema.outputs['result'],
    module_file = tuner_module_file_path,
    train_args = param_tuner_train_args,
    eval_args = param_tuner_eval_args,
    custom_config = param_tuner_custom_config_args
)
context.run(tuner)

Exception ignored in: <function CapturableResource.__del__ at 0x7ff83d2911f0>
Traceback (most recent call last):
  File "/home/jovyan/.local/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py", line 277, in __del__
    self._destroy_resource()
  File "/home/jovyan/.local/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/home/jovyan/.local/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 924, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/jovyan/.local/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3022, in __call__
    filtered_flat_args) = self._maybe_define_function(args, kwargs)
  File "/home/jovyan/.local/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3444, in _maybe_define_function
    graph_function = self._create_graph_function(args, kwargs)
  File "/home/jovyan/.local/l

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age_xf (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
education-num_xf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-gain_xf (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-loss_xf (InputLayer)    [(None, 1)]          0                                            
______________________________________________________________________________________________

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age_xf (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
education-num_xf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-gain_xf (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-loss_xf (InputLayer)    [(None, 1)]          0                                            
______________________________________________________________________________________________

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age_xf (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
education-num_xf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-gain_xf (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-loss_xf (InputLayer)    [(None, 1)]          0                                            
______________________________________________________________________________________________

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age_xf (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
education-num_xf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-gain_xf (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-loss_xf (InputLayer)    [(None, 1)]          0                                            
______________________________________________________________________________________________

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age_xf (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
education-num_xf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-gain_xf (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-loss_xf (InputLayer)    [(None, 1)]          0                                            
______________________________________________________________________________________________

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age_xf (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
education-num_xf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-gain_xf (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-loss_xf (InputLayer)    [(None, 1)]          0                                            
______________________________________________________________________________________________

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age_xf (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
education-num_xf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-gain_xf (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-loss_xf (InputLayer)    [(None, 1)]          0                                            
______________________________________________________________________________________________

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age_xf (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
education-num_xf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-gain_xf (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-loss_xf (InputLayer)    [(None, 1)]          0                                            
______________________________________________________________________________________________

ExecutionResult(
    component_id: Tuner
    execution_id: 12
    outputs:
        best_hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 19
        type_id: 29
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Tuner/best_hyperparameters/12"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 29
        name: "HyperParameters"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [52]:
bucket.upload_file(os.path.join(tuner.outputs['best_hyperparameters'].get()[0].uri, 'best_hyperparameters.txt'),
                   'tuner/best_hyperparameters.txt')

In [53]:
import_tuner = tfx.dsl.Importer(
    source_uri = '%s/tuner/' % bucket_name_s3_prefix,
    artifact_type = tfx.types.standard_artifacts.HyperParameters).with_id('tuner_importer')
context.run(import_tuner)

ExecutionResult(
    component_id: tuner_importer
    execution_id: 13
    outputs:
        result: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 20
        type_id: 29
        uri: "s3://census-income/tuner/"
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        , artifact_type: id: 29
        name: "HyperParameters"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [54]:
trainer_module_file_name = 'adult_income_trainer_module.py'
trainer_module_file_path = os.path.join(os.getcwd(), trainer_module_file_name)

In [55]:
%%writefile {trainer_module_file_path}

import absl
import os
import sys

from typing import List, Text

import kerastuner
import tensorflow as tf
import tensorflow_transform as tft

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, concatenate

from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx.components.trainer.executor import TrainerFnArgs
from tfx_bsl.tfxio import dataset_options

NUMERIC_FEATURE_KEYS = [
    'age',
    'education-num',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
]
ONE_HOT_FEATURES = {'workclass': 8,
                    'education': 16,
                    'marital-status': 7,
                    'occupation': 14,
                    'relationship': 6,
                    'gender': 2
                   }
LABEL_KEY = 'income'

def transformed_name(key):
    return key + '_xf'

def input_fn(file_pattern: List[Text],
             data_accessor: DataAccessor,
             tf_transform_output: tft.TFTransformOutput,
             batch_size: int = 200) -> tf.data.Dataset:
    return data_accessor.tf_dataset_factory(
        file_pattern,
        dataset_options.TensorFlowDatasetOptions(
            batch_size=batch_size, label_key=transformed_name(LABEL_KEY)),
        tf_transform_output.transformed_metadata.schema)

def create_model(hparams: kerastuner.HyperParameters):
    model = Sequential()

    inputs = []
    for key in NUMERIC_FEATURE_KEYS:
        inputs.append(Input(shape=(1), name=transformed_name(key)))
    for key, dim in ONE_HOT_FEATURES.items():
        for i in range(0, dim):
            inputs.append(Input(shape=(1), name=transformed_name(key + '_' + str(i))))

    outputs = concatenate(inputs)
    units = hparams.get('units')
    outputs = Dense(units, activation='relu')(outputs)
    outputs = Dense(units, activation='relu')(outputs)
    outputs = Dense(1, activation='sigmoid')(outputs)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=hparams.get('optimizer'), loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def get_tf_examples_serving_signature(model, tf_transform_output):
    model.tft_layer_inference = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
    ])
    def serve_tf_examples_fn(serialized_tf_example):
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_feature_spec.pop(LABEL_KEY)
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
        transformed_features = model.tft_layer_inference(raw_features)

        outputs = model(transformed_features)
        return {'outputs': outputs}

    return serve_tf_examples_fn

def get_transform_features_signature(model, tf_transform_output):
    model.tft_layer_eval = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
    ])
    def transform_features_fn(serialized_tf_example):
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
        transformed_features = model.tft_layer_eval(raw_features)
        return transformed_features

    return transform_features_fn

def run_fn(fn_args: TrainerFnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    train_dataset = input_fn(fn_args.train_files, fn_args.data_accessor,
                                tf_transform_output, 40)
    eval_dataset = input_fn(fn_args.eval_files, fn_args.data_accessor,
                               tf_transform_output, 40)

    hparams = kerastuner.HyperParameters.from_config(fn_args.hyperparameters)
    absl.logging.info('HyperParameters for training: %s' % hparams.get_config())

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = fn_args.model_run_dir, update_freq='batch'
    )

    model = create_model(hparams)
    model.fit(
        train_dataset,
        epochs = fn_args.custom_config["epoch"],
        steps_per_epoch = fn_args.train_steps,
        validation_data = eval_dataset,
        validation_steps = fn_args.eval_steps,
        callbacks = [tensorboard_callback]
    )

    signatures = {
        'serving_default':
            get_tf_examples_serving_signature(model, tf_transform_output),
        'transform_features':
            get_transform_features_signature(model, tf_transform_output),
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting /home/jovyan/resource/adult_income_trainer_module.py


In [56]:
bucket.upload_file(trainer_module_file_path, 'module/%s' % trainer_module_file_name)

In [57]:
param_trainer_module_file = '%s/module/%s' % (bucket_name_s3_prefix, trainer_module_file_name)

trainer = tfx.components.Trainer(
    module_file = param_trainer_module_file,
    custom_executor_spec = executor_spec.ExecutorClassSpec(GenericExecutor),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = import_schema.outputs['result'],
    hyperparameters = import_tuner.outputs['result'],
    train_args = param_trainer_train_args,
    eval_args = param_trainer_eval_args,
    custom_config = param_trainer_custom_config_args)
context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age_xf (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
education-num_xf (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-gain_xf (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
capital-loss_xf (InputLayer)    [(None, 1)]          0                                            
____________________________________________________________________________________________

ExecutionResult(
    component_id: Trainer
    execution_id: 14
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 21
        type_id: 26
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Trainer/model/14"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 22
        type_id: 27
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Trainer/model_run/14"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 27
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [58]:
slices = [tfma.SlicingSpec(),
          tfma.SlicingSpec(feature_keys=['gender_0_xf']),
          tfma.SlicingSpec(feature_keys=['age_xf'])
         ]

eval_config = tfma.EvalConfig(
    model_specs = [tfma.ModelSpec(
        label_key='income_xf',
        preprocessing_function_names=['transform_features']
    )],
    slicing_specs = slices,
    metrics_specs = [
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),   #サンプル数
            tfma.MetricConfig(class_name='Precision'),
            tfma.MetricConfig(class_name='Recall'),
            tfma.MetricConfig(class_name='F1Score'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives')
        ])
    ]
)

In [59]:
evaluator = tfx.components.Evaluator(
    examples = example_gen.outputs['examples'],
    model = trainer.outputs['model'],
    eval_config = eval_config)
context.run(evaluator)

Exception ignored in: <function CapturableResource.__del__ at 0x7ff83d2911f0>
Traceback (most recent call last):
  File "/home/jovyan/.local/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py", line 277, in __del__
    self._destroy_resource()
  File "/home/jovyan/.local/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/home/jovyan/.local/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 924, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/jovyan/.local/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3022, in __call__
    filtered_flat_args) = self._maybe_define_function(args, kwargs)
  File "/home/jovyan/.local/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3444, in _maybe_define_function
    graph_function = self._create_graph_function(args, kwargs)
  File "/home/jovyan/.local/l

ExecutionResult(
    component_id: Evaluator
    execution_id: 15
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 23
        type_id: 31
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Evaluator/evaluation/15"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 24
        type_id: 32
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Evaluator/blessing/15"
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [60]:
fairness_eval_config = tfma.EvalConfig(
    model_specs = [tfma.ModelSpec(
        label_key='income_xf',
        preprocessing_function_names=['transform_features']
    )],
    slicing_specs = slices,
    metrics_specs = [
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),   #サンプル数
            tfma.MetricConfig(class_name='Precision'),
            tfma.MetricConfig(class_name='Recall'),
            tfma.MetricConfig(class_name='F1Score'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='FairnessIndicators',
                             config='{"thresholds": [0.25, 0.5, 0.75]}')
        ])
    ]
)

In [61]:
fairness_evaluator = tfx.components.Evaluator(
    examples = example_gen.outputs['examples'],
    model = trainer.outputs['model'],
    eval_config = fairness_eval_config)
context.run(fairness_evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 16
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 25
        type_id: 31
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Evaluator/evaluation/16"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 26
        type_id: 32
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Evaluator/blessing/16"
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [62]:
blessing_eval_config = tfma.EvalConfig(
    model_specs = [tfma.ModelSpec(
        label_key='income_xf'
    )],
    slicing_specs = [tfma.SlicingSpec()],
    metrics_specs = [
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='AUC',
                              threshold=tfma.MetricThreshold(
                                  value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.70})
                              ))
        ])
    ]
)

In [63]:
blessing_evaluator = tfx.components.Evaluator(
    examples = transform.outputs['transformed_examples'],
    model = trainer.outputs['model'],
    eval_config = blessing_eval_config)
context.run(blessing_evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 17
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 27
        type_id: 31
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Evaluator/evaluation/17"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 28
        type_id: 32
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Evaluator/blessing/17"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Trainer/model/14"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 21
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [64]:
blessing_result = blessing_evaluator.outputs['evaluation'].get()[0].uri
tfma.load_validation_result(blessing_result)

validation_ok: true
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 1
  }
}

In [65]:
param_push_destination = {"filesystem": {"base_directory": "%s/serving_model" % bucket_name_s3_prefix}}

In [66]:
from tfx.proto import  pusher_pb2
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
        push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory='s3://census-income/serving_model')))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 18
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 29
        type_id: 34
        uri: "/tmp/tfx-interactive-2022-05-23T05_14_41.408365-a72qo05b/Pusher/pushed_model/18"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "s3://census-income/1653283269"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1653283269"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 34
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [67]:
from tfx.orchestration.kubeflow import kubeflow_dag_runner

metadata_config = kubeflow_dag_runner.get_default_kubeflow_metadata_config()
metadata_config.mysql_db_service_host.value = 'mysql.kubeflow'
metadata_config.mysql_db_service_port.value = "3306"
metadata_config.mysql_db_name.value = "metadb"
metadata_config.mysql_db_user.value = "root"
metadata_config.mysql_db_password.value = ""
metadata_config.grpc_config.grpc_service_host.value = 'metadata-grpc-service.kubeflow.svc.cluster.local'
metadata_config.grpc_config.grpc_service_port.value = '8080'

In [68]:
def env_params(name='S3_ENDPOINT', value=''):
    def _env_params(task):
        from kubernetes import client as k8s_client
        return(task.add_env_variable(k8s_client.V1EnvVar(name=name, value=value)))
    return _env_params

operator_funcs = []
operator_funcs.append(env_params('AWS_ACCESS_KEY_ID', os.environ['AWS_ACCESS_KEY_ID']))
operator_funcs.append(env_params('AWS_SECRET_ACCESS_KEY', os.environ['AWS_SECRET_ACCESS_KEY']))
operator_funcs.append(env_params('AWS_REGION', os.environ["AWS_REGION"]))
operator_funcs.append(env_params('S3_ENDPOINT', os.environ["S3_ENDPOINT"]))
operator_funcs.append(env_params('S3_USE_HTTPS', '0'))
operator_funcs.append(env_params('S3_VERIFY_SSL', '0'))

In [69]:
# boto3とkeras-tunerがインストールされたDockerImageを使用
tfx_image = 'nrt.ocir.io/orasejapan/ocha_demo'

pipeline_config = kubeflow_dag_runner.KubeflowDagRunnerConfig(tfx_image = tfx_image,
                                            pipeline_operator_funcs = operator_funcs,
                                            kubeflow_metadata_config = metadata_config)

In [70]:
import json

# RuntimeParameter としてパイプライン実行時のパラメータとする
# ExampleGenの入力ファイル
pipeline_example_gen_input = tfx.dsl.experimental.RuntimeParameter(
    name='input_config', ptype=str, default=json.dumps(example_gen_input))
example_gen.exec_properties['input_config'] = pipeline_example_gen_input

# Pusherのモデルアップロード先
pipeline_push_destination = tfx.dsl.experimental.RuntimeParameter(
    name='push_destination', ptype=str, default=json.dumps(param_push_destination))
pusher.exec_properties['push_destination'] = pipeline_push_destination

In [71]:
components = [
    example_gen,
    statistics_gen,
    import_schema,
    example_validator,
    transform,
    import_tuner,
    trainer,
    evaluator,
    pusher,
]

In [72]:
pipeline_name = 'tfx_census_income_pipeline'

pipeline = tfx.dsl.Pipeline(
    pipeline_name = pipeline_name,
    pipeline_root = '%s/pipelines/%s' % (bucket_name_s3_prefix, pipeline_name),
    components = components,
    enable_cache = True,
    beam_pipeline_args = beam_pipeline_args)

In [73]:
package_dir = os.path.join(os.getcwd(), "package")
os.makedirs(package_dir, exist_ok=True)
package_file_name = '%s_pipeline.tar.gz' % pipeline_name
package_path = os.path.join(package_dir, package_file_name)

kubeflow_dag_runner.KubeflowDagRunner(output_dir=package_dir,
                                      output_filename=package_file_name,
                                      config=pipeline_config).run(pipeline)